In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout,Activation
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from IPython import display
from PIL import Image
import os
import numpy as np
from keras.preprocessing import image

Using Theano backend.


In [34]:
def createCNN(classes,cnnOutput=(32,3,3), poolSize=(2,2)):
    """
    :classes: (int) number of categories
    """
    classifier = Sequential()
    classifier.add(Convolution2D(cnnOutput[0],cnnOutput[1],cnnOutput[2], input_shape=(64,64,3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=poolSize))
    classifier.add(Flatten())
    classifier.add(Dense(output_dim=classes, activation='relu'))
    classifier.add(Dense(output_dim=classes, activation='sigmoid'))
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
    
    return classifier

In [11]:
def sperateData():
    import os
    # for folder in os.listdir('images/train'):
    #     os.mkdir('images/test/'+folder)
    import shutil
    import numpy as np
    for folder in os.listdir('images/train'):
        for image in os.listdir('images/train/'+folder):
            if np.random.rand(1)<0.2:
                shutil.move('images/train/'+folder+'/'+image, 'images/test/'+folder+'/'+image)
for folder in os.listdir('images/train'):
    

In [15]:
def getDataset(classes,train_source,test_source):
    trainGen = ImageDataGenerator(
    rescale = 1.0/255,
    shear_range = 0.1,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip=True
    )
    
    testGen = ImageDataGenerator(rescale = 1.0/255)
    
    trainSet = trainGen.flow_from_directory(train_source, target_size=(96,96), batch_size=32, class_mode='categorical')
    testSet = testGen.flow_from_directory(test_source, target_size=(96,96), batch_size=32, class_mode='categorical')
    return trainSet, trainSet

In [18]:
def makeModel(num_classes):
    classifier = Sequential()
    # First section
    classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding="same", input_shape=(96,96,3), activation="relu" ))
    classifier.add(BatchNormalization(axis=1))
    classifier.add(MaxPooling2D(pool_size=(3,3)))
    classifier.add(Dropout(0.25))
    
    #Second section
    classifier.add(Convolution2D(filters=64, kernel_size=(3,3),padding="same", activation="relu" ))
    classifier.add(BatchNormalization(axis=1))
    classifier.add(Convolution2D(filters=64, kernel_size=(3,3),padding="same", activation="relu" ))
    classifier.add(BatchNormalization(axis=1))
    classifier.add(MaxPooling2D(pool_size=(2,2)))
    classifier.add(Dropout(0.25))
    
    #Third section
    classifier.add(Convolution2D(filters=128, kernel_size=(3,3),padding="same", activation="relu" ))
    classifier.add(BatchNormalization(axis=1))
    classifier.add(Convolution2D(filters=128, kernel_size=(3,3),padding="same", activation="relu" ))
    classifier.add(BatchNormalization(axis=1))
    classifier.add(MaxPooling2D(pool_size=(2,2)))
    classifier.add(Dropout(0.25))
    
    #Final Fully connected --> Relu layer
    classifier.add(Flatten())
    classifier.add(Dense(units=1024, activation="relu"))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.5))
    
    classifier.add(Dense(units=num_classes, activation="sigmoid"))
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
    return classifier
    

In [19]:
train_dir = 'images/train/'
test_dir = 'images/test/'
num_classes = len(os.listdir('images/train'))
classifier = makeModel(num_classes)
trainSet, testSet = getDataset(classes=num_classes, train_source=train_dir, test_source=test_dir)

Found 7191 images belonging to 9 classes.
Found 1809 images belonging to 9 classes.


In [ ]:
classifier.fit_generator(trainSet,steps_per_epoch=8000,epochs=10, validation_data=testSet, validation_steps=800)

Epoch 1/10


In [ ]:
classifier.save('models/my_model.h5')

In [4]:
from keras.models import load_model
classifier = load_model('models/my_model.h5')

In [19]:
test_image = image.load_img('images/test/french_fries/1132539.jpg',target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)

In [27]:
classifier = createCNN(num_classes)

/home/ubuntu/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  
/home/ubuntu/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=9)`
  if __name__ == '__main__':
/home/ubuntu/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=9)`
  # Remove the CWD from sys.path while we load stuff.


In [29]:
res=(classifier.predict(test_image))

In [30]:
np.argmax(res)

4

In [31]:
res

array([[5.9401594e-12, 8.5529870e-01, 3.0599590e-04, 2.5342540e-06,
        1.0000000e+00, 1.0000000e+00, 2.5837596e-11, 9.0986669e-01,
        3.4521233e-10]], dtype=float32)

In [32]:
trainSet.class_indices

{'chicken_wings': 0,
 'chocolate_cake': 1,
 'dumplings': 2,
 'french_fries': 3,
 'fried_rice': 4,
 'hamburger': 5,
 'ice_cream': 6,
 'pizza': 7,
 'ramen': 8}

In [12]:
c = makeModel(9)

In [13]:
print(c.trainable_weights, (
    np.sum([K.count_params(p) for p in set(c.trainable_weights)])))

[conv2d_6/kernel, conv2d_6/bias, batch_normalization_7/gamma, batch_normalization_7/beta, conv2d_7/kernel, conv2d_7/bias, batch_normalization_8/gamma, batch_normalization_8/beta, conv2d_8/kernel, conv2d_8/bias, batch_normalization_9/gamma, batch_normalization_9/beta, dense_3/kernel, dense_3/bias, batch_normalization_10/gamma, batch_normalization_10/beta, dense_4/kernel, dense_4/bias] 8494441
